# Building our First RAG bot - Skill: talk to Search Engine

We have now all the building blocks to build our first Bot that "talks with my data". These blocks are:

1) A well indexed hybrid (text and vector) engine with my data in chunks -> Azure AI Search
2) A good LLM python framework to build LLM Apps -> LangChain
3) Quality OpenAI GPT models that understand language and follow instructions
4) A persisten memory database -> CosmosDB

We are missing just one thing: **Agents**.

In this Notebook we introduce the concept of Agents and we use it to build or first RAG bot.

In [1]:
import os
import random
import asyncio
from typing import Dict, List
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Type

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField, ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

#custom libraries that we will use later in the app
from common.utils import  GetDocSearchResults_Tool
from common.prompts import AGENT_DOCSEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

from dotenv import load_dotenv
load_dotenv("credentials.env")


True

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introducing: Agents

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I build a smart bot that talks to my data", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use 'experts/tools' in order to read/load/understand/interact with a any particular source of data.

Let's create then an Agent that interact with the user and uses a Tool to get the information from the Search engine.

#### 1. We start first defining the Tool/Expert

Tools are functions that an agent can invoke. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it. If you don't describe the tools well, the agent won't know how to use them properly.

In [3]:
index1_name = "srch-index-files"
index2_name = "srch-index-csv"
index3_name = "srch-index-books"
indexes = [index1_name, index2_name, index3_name]

We have to convert the Retreiver object into a Tool object ("the expert"). Check out the Tool `GetDocSearchResults_Tool` in `utils.py` and see how it is done.

Declare the tools the agent will use

In [4]:
tools = [GetDocSearchResults_Tool(indexes=indexes, k=10, reranker_th=1, sas_token=os.environ['BLOB_SAS_TOKEN'])]

#### 2. Define the LLM to use

In [5]:
COMPLETION_TOKENS = 1500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4oMINI_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True)

#### 3. Bind tools to the LLM

Newer OpenAI models (1106 and newer) have been fine-tuned to detect when one or more function(s) should be called and respond with the inputs that should be passed to the function(s). In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call these functions. The goal of the OpenAI tools APIs is to more reliably return valid and useful function calls than what can be done using a generic text completion or chat API.

OpenAI termed the capability to invoke a single function as **functions**, and the capability to invoke one or more functions as [**tools**](https://platform.openai.com/docs/guides/function-calling).

> OpenAI API has deprecated functions in favor of tools. The difference between the two is that the tools API allows the model to request that multiple functions be invoked at once, which can reduce response times in some architectures. It’s recommended to use the tools agent for OpenAI models.

Having an LLM call multiple tools at the same time can greatly speed up agents whether there are tasks that are assisted by doing so. Thankfully, OpenAI models versions 1106 and newer support parallel function calling, which we will need to make sure our smart bot is performant.

##### **From now on and for the rest of the notebooks, we are going to use OpenAI tools API tool call our experts/tools**

To pass in our tools to the agent, we just need to format them to the [OpenAI tool format](https://platform.openai.com/docs/api-reference/chat/create) and pass them to our model. (By bind-ing the functions, we’re making sure that they’re passed in each time the model is invoked.)

In [6]:
# Bind (attach) the tools/functions we want on each LLM call

llm_with_tools = llm.bind_tools(tools)

# Let's also add the option to configure in real time the model we want

llm_with_tools = llm_with_tools.configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt4omini",
    gpt4o=AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True) 
)

#### 4. Define the System Prompt

Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format. We will just have two input variables: `question` and `agent_scratchpad`. The input variable `question` should be a string containing the user objective, and `agent_scratchpad` should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

Get the prompt to use `AGENT_DOCSEARCH_PROMPT` - you can modify this in `prompts.py`! Check it out!
It looks like this:

```python
AGENT_DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", CUSTOM_CHATBOT_PREFIX  + DOCSEARCH_PROMPT_TEXT),
        MessagesPlaceholder(variable_name='history', optional=True),
        ("human", "{question}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)
```

In [7]:
prompt = AGENT_DOCSEARCH_PROMPT

#### 5. Create the agent

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [8]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

Or , which is equivalent, LangChain has a class that does exactly the cell code above: `create_openai_tools_agent`

```python
agent = create_openai_tools_agent(llm, tools, prompt)
```

**Important Note: Other models like Mistral Large or Command R+ won't work with the same OpenAI Tools API, so in order to create agents with these models, try using the ReAct type instead from langchain**. Like [THIS COHERE AGENT](https://python.langchain.com/docs/integrations/providers/cohere/#react-agent) for example

Create an agent executor by passing in the agent and tools

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Give it memory - since AgentExecutor is also a Runnable class, we do the same with did on Notebook 5

In [10]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

Because cosmosDB needs two fields (an id and a partition), and RunnableWithMessageHistory takes by default only one identifier for memory (session_id), we need to use `history_factory_config` parameter and define the multiple keys for the memory class

In [11]:
userid_spec = ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        )
session_id = ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        )

In [12]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[userid_spec,session_id]
)

In [13]:
# configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
config

{'configurable': {'session_id': 'session931', 'user_id': 'user627'}}

#### 6.Run the Agent!

In [14]:
%%time
agent_with_chat_history.invoke({"question": "How Chandler proposes to Monica?"}, config=config)

CPU times: user 320 ms, sys: 19.7 ms, total: 340 ms
Wall time: 5.42 s


{'question': 'How Chandler proposes to Monica?',
 'history': [],
 'output': 'Chandler Bing\'s proposal to Monica Geller is a memorable moment in the series *Friends*. It occurs in Season 6, Episode 24, titled "The One with the Proposal." Here’s how it unfolds:\n\n1. **Setting the Scene**: Chandler decides to propose to Monica in a romantic manner. He takes her to a restaurant, intending to make the moment special by ordering her favorite champagne.\n\n2. **The Proposal**: As they sit down, Chandler starts to express his feelings. He realizes that what matters most is not the setting or the words, but the love he feels for Monica. He says:\n   > "I thought that it mattered what I said or where I said it. Then I realized the only thing that matters is that you make me happier than I ever thought I could be. And if you\'ll let me, I will spend the rest of my life trying to make you feel the same way. Monica, will you marry me?"\n\n3. **Monica\'s Response**: Overwhelmed with joy, Monica re

In [15]:
printmd(agent_with_chat_history.invoke(
    {"question": "What are markov chains and is there an application in medicine?"}, 
    config=config)["output"])

## What are Markov Chains?

**Markov Chains** are mathematical models that describe systems that transition from one state to another within a finite or countable number of states. The key characteristic of a Markov Chain is the **Markov property**, which states that the future state of the system depends only on the current state and not on the sequence of events that preceded it. This property allows for the simplification of complex systems into manageable models.

### Key Components:
1. **States**: The possible situations in which the system can exist.
2. **Transition Probabilities**: The probabilities of moving from one state to another.
3. **Transition Matrix**: A matrix that represents the probabilities of transitioning from each state to every other state.

Markov Chains can be classified into:
- **Discrete-time Markov Chains**: Where transitions occur at fixed time intervals.
- **Continuous-time Markov Chains**: Where transitions can occur at any time.

### Applications in Medicine
Markov Chains have various applications in the medical field, particularly in modeling disease progression, treatment outcomes, and healthcare decision-making.

1. **Disease Spread Modeling**: Markov Chains can be used to model the spread of infectious diseases, such as COVID-19. For instance, a nonlinear Markov chain model was proposed to analyze the behavior of the COVID-19 pandemic, estimating daily new cases and examining correlations with daily deaths [[8]](http://medrxiv.org/cgi/content/short/2020.04.21.20073668v1?rss=1?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

2. **Epidemiological Simulation**: A discrete-time Markov chain simulator has been developed to model the dynamics of epidemics, allowing researchers to test different control algorithms [[9]](https://doi.org/10.1109/embc.2016.7591271).

3. **Cost-Effectiveness Analysis**: Markov models are employed to assess the cost-effectiveness of medical treatments. For example, a study used a Markov model to evaluate the cost-effectiveness of extracorporeal cardiopulmonary resuscitation (ECPR) for cardiac arrest patients [[10]](https://doi.org/10.1016/j.resuscitation.2019.08.024).

4. **Airborne Disease Transmission**: Markov chains have been integrated with computational fluid dynamics to predict airborne disease transmission in enclosed environments, providing valuable insights for reducing infection risks [[6]](https://doi.org/10.1111/ina.12056).

These applications illustrate the versatility of Markov Chains in addressing complex medical problems, enhancing our understanding of disease dynamics, and informing healthcare decisions.

In [16]:
printmd(agent_with_chat_history.invoke(
        {"question": "Interesting, Tell me more about the use of markov chains, specifically in the spread of viruses"},
        config=config)["output"])

Markov chains have been effectively utilized in modeling the spread of viruses, particularly in understanding how infectious diseases propagate through populations. Here are some key insights into their application:

### 1. Spatial Markov Chain Models
A **Spatial Markov Chain model** represents the spread of viruses by connecting nodes that symbolize individuals (e.g., humans). The edges between these nodes signify interpersonal relationships. The likelihood of virus transmission is influenced by the intensity of contact between individuals, and the transfer of infection is determined by chance. This model can be extended to simulate various lockdown scenarios, helping to assess the impact of social distancing measures on virus spread [[1]](https://arxiv.org/pdf/2004.05635v1.pdf?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

### 2. Continuous-Time Markov Chain (CTMC) Models
CTMC models have been applied to study the emergence and re-emergence of infectious diseases. These models account for different groups within a population, such as superspreaders—individuals who infect a disproportionately large number of others. The transmission rates can vary based on whether the host is infectious or susceptible. This approach allows researchers to estimate the probability of minor or major epidemics based on initial conditions [[2]](https://doi.org/10.1080/17513758.2018.1538462; https://www.ncbi.nlm.nih.gov/pubmed/30381000/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

### 3. Nonlinear Markov Chain Models
During the COVID-19 pandemic, nonlinear Markov chain models were developed to analyze the spread of the virus using data from various countries. These models estimate daily new cases and examine correlations between new cases and deaths, providing valuable insights into epidemic trends [[3]](http://medrxiv.org/cgi/content/short/2020.04.21.20073668v1?rss=1?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

### 4. Parametric Time-Varying Markov Processes
These processes help estimate model parameters and approximate unobserved counts of infected, recovered, and immunized individuals based on daily reported cases and deaths. This approach is particularly useful in scenarios where many infected individuals show no symptoms, complicating the tracking of the virus's spread [[4]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7090511/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

### 5. Non-Markovian Models
Recent studies suggest that the epidemiological parameters for COVID-19 do not always follow exponential distributions, leading to the development of **non-Markovian models**. These models aim to capture more accurately the complex dynamics of virus transmission, especially in the absence of strict control measures [[5]](https://doi.org/10.1101/2020.02.07.20021139?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&sig=0eJm6CaaACeHGfgKGIXE163moq7X0Mu6tbZcCU0MHkA%3D).

### Conclusion
Markov chains, both in their standard and nonlinear forms, provide powerful tools for modeling the spread of viruses. They help researchers understand the dynamics of infectious diseases, estimate the potential for outbreaks, and evaluate the effectiveness of interventions. The evolution of these models continues to adapt to the complexities of real-world scenarios, particularly in light of new infectious diseases like COVID-19.

In [17]:
%%time
printmd(agent_with_chat_history.invoke({"question": "Thhank you!"}, config=config)["output"])

You're welcome! If you have any questions or need assistance with anything else, feel free to ask!

CPU times: user 61.9 ms, sys: 10.3 ms, total: 72.2 ms
Wall time: 1.25 s


### Let's add more things we have learned so far: dynamic LLM selection of GPT4o and asyncronous streaming

In [18]:
agent = create_openai_tools_agent(llm_with_tools.with_config(configurable={"model": "gpt4o"}), tools, prompt) # We select now GPT-4o
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
agent_with_chat_history = RunnableWithMessageHistory(agent_executor,get_session_history,input_messages_key="question", 
                                                     history_messages_key="history", history_factory_config=[userid_spec,session_id])

In prior notebooks with use the function `.stream()` of the runnable in order to stream the tokens. However if you need to stream individual tokens from the agent or surface steps occuring within tools, you would need to use a combination of `Callbacks` and `.astream()` OR the new `astream_events` API (beta).

Let’s use here the astream_events API to stream the following events:

    Agent Start with inputs
    Tool Start with inputs
    Tool End with outputs
    Stream the agent final anwer token by token
    Agent End with outputs

In [19]:
QUESTION = "Tell me more about chandler proposing to monica, search again multiple times and provide a deeper explanation"

In [20]:
async for event in agent_with_chat_history.astream_events(
    {"question": QUESTION}, config=config, version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (event["name"] == "AgentExecutor"):
            print( f"Starting agent: {event['name']}")
    elif kind == "on_chain_end":
        if (event["name"] == "AgentExecutor"):  
            print()
            print("--")
            print(f"Done agent: {event['name']}")
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        # Empty content in the context of OpenAI means that the model is asking for a tool to be invoked.
        # So we only print non-empty content
        if content:
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        # print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: AgentExecutor
--
Starting tool: docsearch with inputs: {'query': 'Chandler proposes to Monica scene'}
--
Starting tool: docsearch with inputs: {'query': 'Chandler Bing proposal to Monica Geller details'}
--
Starting tool: docsearch with inputs: {'query': 'Friends Chandler Monica proposal episode'}
Done tool: docsearch
--
Done tool: docsearch
--
Done tool: docsearch
--
Chandler Bing's proposal to Monica Geller is a pivotal moment in the series *Friends*, and it unfolds with a mix of romance, humor, and heartfelt emotion.

### The Build-Up

Chandler initially plans to propose to Monica in a traditional manner by taking her to her favorite restaurant. He intends to order her favorite champagne, which she knows is expensive, and then propose a toast that turns into a marriage proposal [[4]](https://blobstorageuq7x4ufcftcpm.blob.core.windows.net/friends/s06/e24/c01.txt?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-10-03T01:44:00Z&st=2024-10-02T17:44:00Z&spr=https&si

# Summary

We just built our first RAG BOT!.

- We learned that **Agents + Tools are the best way to go about building Bots**. <br>
- We converted the Azure Search retriever into a Tool using the function `GetDocSearchResults_Tool` in `utils.py`
- We learned about the events API, one way to stream the answer from agents


# NEXT

Now that we have a bot with one skill (Document Search), let's build more skills!. In the next Notebook, we are going to build an agent that can understand tabular data in csv file and can execute python commands